# Вебинар 6. Двухуровневые модели рекомендаций


# Import libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

## Read data

In [2]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# Process features dataset

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

In [4]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [5]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [6]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [7]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [8]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [9]:
# выше видим разброс по пользователям и товарам

In [10]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


# Prefilter items

In [11]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\Nata\Desktop\webinar_6\webinar_6\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001


# Make cold-start to warm-start

In [12]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


In [13]:
# Теперь warm-start по пользователям

# Init/train recommender

In [14]:
recommender = MainRecommender(data_train_matcher)

### Варианты, как получить кандидатов

# Eval recall of matching

### Измеряем recall@k

Это будет в ДЗ: 

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [15]:
ACTUAL_COL = 'actual'

In [16]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [17]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
for N_PREDICT in (20, 50, 100, 200, 500):
    result_eval_matcher[f'own_rec_{N_PREDICT}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
    result_eval_matcher[f'sim_item_rec_{N_PREDICT}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
    result_eval_matcher[f'als_rec_{N_PREDICT}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))
    
result_eval_matcher.head(2)

Wall time: 2min 42s


,user_id,actual,own_rec_20,sim_item_rec_20,als_rec_20,own_rec_50,sim_item_rec_50,als_rec_50,own_rec_100,sim_item_rec_100,als_rec_100,own_rec_200,sim_item_rec_200,als_rec_200,own_rec_500,sim_item_rec_500,als_rec_500
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[884686, 1007512, 9297615, 5577022, 9803545, 9...","[885290, 5572884, 1088387, 960732, 1104349, 10...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[884686, 1007512, 9297615, 5577022, 9803545, 9...","[885290, 5572884, 1088387, 960732, 1104349, 10...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[884686, 1007512, 9297615, 5577022, 9803545, 9...","[885290, 5572884, 1088387, 960732, 1104349, 10...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[884686, 1007512, 9297615, 5577022, 9803545, 9...","[885290, 5572884, 1088387, 960732, 1104349, 10...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[884686, 1007512, 9297615, 5577022, 9803545, 9...","[885290, 5572884, 1088387, 960732, 1104349, 10..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 897954, 880888, 81...","[5569230, 5569845, 986021, 8090521, 8090537, 1...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 897954, 880888, 81...","[5569230, 5569845, 986021, 8090521, 8090537, 1...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 897954, 880888, 81...","[5569230, 5569845, 986021, 8090521, 8090537, 1...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 897954, 880888, 81...","[5569230, 5569845, 986021, 8090521, 8090537, 1...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 1044078, 897954, 880888, 81...","[5569230, 5569845, 986021, 8090521, 8090537, 1..."


In [18]:
%%time
#result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=N_PREDICT))

Wall time: 0 ns


### Пример оборачивания

In [19]:
# # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [20]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [21]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

### Recall@K of matching

In [22]:
for TOPK_RECALL in (20, 50, 100, 200, 500):
    print(f'TOPK_RECALL = {TOPK_RECALL}:\n{sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)}')

TOPK_RECALL = 20:
[('own_rec_20', 0.03928427679372909), ('own_rec_50', 0.03928427679372909), ('own_rec_100', 0.03928427679372909), ('own_rec_200', 0.03928427679372909), ('own_rec_500', 0.03928427679372909), ('als_rec_20', 0.029810257443612417), ('als_rec_50', 0.029810257443612417), ('als_rec_100', 0.029810257443612417), ('als_rec_200', 0.029810257443612417), ('als_rec_500', 0.029810257443612417), ('sim_item_rec_20', 0.017207314017571328), ('sim_item_rec_50', 0.017207314017571328), ('sim_item_rec_100', 0.017207314017571328), ('sim_item_rec_200', 0.017207314017571328), ('sim_item_rec_500', 0.017207314017571328)]
TOPK_RECALL = 50:
[('own_rec_50', 0.06525657038145175), ('own_rec_100', 0.06525657038145175), ('own_rec_200', 0.06525657038145175), ('own_rec_500', 0.06525657038145175), ('als_rec_50', 0.04740565101714787), ('als_rec_100', 0.04740565101714787), ('als_rec_200', 0.04740565101714787), ('als_rec_500', 0.04740565101714787), ('own_rec_20', 0.03928427679372909), ('sim_item_rec_50', 0.03

При увеличении количества предсказаний N_PREDICT и TOPK_RECALL, метрика RECALL увеличивается. Так как это приводит к увеличению обучающей матрицы модели второго уровня, выбирая эту размерность, нужно исходить из мощности оборудования.

### Precision@5 of matching

In [23]:
TOPK_PRECISION = 5

In [24]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec_20', 0.17712691771268974),
 ('own_rec_50', 0.17712691771268974),
 ('own_rec_100', 0.17712691771268974),
 ('own_rec_200', 0.17712691771268974),
 ('own_rec_500', 0.17712691771268974),
 ('als_rec_20', 0.11864249186424826),
 ('als_rec_50', 0.11864249186424826),
 ('als_rec_100', 0.11864249186424826),
 ('als_rec_200', 0.11864249186424826),
 ('als_rec_500', 0.11864249186424826),
 ('sim_item_rec_20', 0.06211064621106492),
 ('sim_item_rec_50', 0.06211064621106492),
 ('sim_item_rec_100', 0.06211064621106492),
 ('sim_item_rec_200', 0.06211064621106492),
 ('sim_item_rec_500', 0.06211064621106492)]

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

In [25]:
# 3 временных интервала
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

## Подготовка данных для трейна

In [26]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [27]:
# собираем кандитатов с первого этапа (matcher)
N_PREDICT = 50
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [28]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [29]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [30]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [31]:
df_match_candidates.head(2)

,user_id,item_id
0,2070,1105426
0,2070,1097350


### Check warm start

In [32]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (107550, 2) Users: 2151 Items: 4574


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [33]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

In [34]:
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


#### Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей

In [35]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [36]:
df_ranker_train.target.value_counts()

0.0    99177
1.0     7795
Name: target, dtype: int64

In [37]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


(!) На каждого юзера 50 item_id-кандидатов

In [38]:
df_ranker_train['target'].mean()

0.07286953595333358

## Подготавливаем фичи для обучения модели

In [39]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [40]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [41]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее кол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

In [42]:
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [43]:
%%time
# Создадим словари для создания новых признаков:
# Средний чек
check_mean = data_train_matcher.groupby(['user_id','basket_id']).sales_value.mean()
check_mean = {user:check_mean[user].mean() for user in data_train_matcher.user_id.unique()}

# Кол-во покупок в каждой категории
commodity = data_train_matcher.merge(item_features[['item_id','commodity_desc']], on='item_id', how='left').fillna('999999').groupby(['user_id','commodity_desc'])['item_id'].count()
commodity = {user: commodity[user].to_dict() for user in data_train_matcher.user_id.unique()}

# цена item_id
temp = data_train_matcher.groupby('item_id')['quantity','sales_value','retail_disc'].mean()
price_dict = ((temp.sales_value + abs(temp.retail_disc))/temp.quantity).to_dict()

# Кол-во покупок в неделю
data_train_matcher.loc[data_train_matcher.week_no > 52, 'week_no'] = data_train_matcher.loc[data_train_matcher.week_no > 52, 'week_no'] - 52
item_week_dict = data_train_matcher.groupby(['item_id','week_no'])['quantity'].sum()
item_week_dict = {item:item_week_dict[item].to_dict() for item in data_train_matcher.item_id.unique()}

#(Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
# значение будет некорректным, т.к.у большой части товаров установили item_id=999999 и не можем определить категорию
user_item_week_ = pd.DataFrame(data_train_matcher[['user_id','item_id','quantity','week_no']]).merge(item_features[['item_id','commodity_desc']], on='item_id', how='left').fillna('999999')
u_i_week = user_item_week_.groupby(['commodity_desc','week_no','user_id'])['quantity'].sum()

category_list = user_item_week_.commodity_desc.unique()
u_i_week_dict = {cat: {week:u_i_week[cat,week].to_dict() for week in range(1,53) if (cat,week) in u_i_week.index} for cat in category_list}

<timed exec>:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Wall time: 1min 2s


In [44]:
def user_cat_week_sub(week, user, category, dictionary):
    
    if week in dictionary[category].keys():
        if user in dictionary[category][week].keys():
            return dictionary[category][week][user]-sum(dictionary[category][week].values())/sum(dictionary[category][week].keys())
        else:
            return -sum(dictionary[category][week].values())/sum(dictionary[category][week].keys())
    else:
        return 0

In [45]:
def user_cat_week_div(week, user, category, dictionary):
    
    if week in dictionary[category].keys():
        if user in dictionary[category][week].keys():
            return dictionary[category][week][user]/sum(dictionary[category][week].values())/sum(dictionary[category][week].keys())
        else:
            return 0
    else:
        return 0

In [46]:
def del_special_char(string):
    return ''.join(ch if ch.isalnum() else '_' for ch in string.lower())

In [47]:
%%time
# Средний чек
df_ranker_train['check_mean'] = df_ranker_train.apply(lambda x: check_mean[x[0]] ,axis=1)

# Кол-во покупок в каждой категории
temp = pd.DataFrame(df_ranker_train.user_id).drop_duplicates()
for cat in category_list:
    temp['cat_'+del_special_char(cat)] = [commodity[user][cat] if cat in commodity[user].keys() else 0 for user in temp.user_id]
df_ranker_train = df_ranker_train.merge(temp, on='user_id')

# цена item_id
df_ranker_train['price'] = df_ranker_train.apply(lambda x: price_dict[x[1]] ,axis=1)

# Кол-во покупок в неделю
temp = pd.DataFrame(df_ranker_train.item_id).drop_duplicates()
for week in range(1,53):
    temp['w_'+str(week)] = [item_week_dict[item][week] if week in item_week_dict[item].keys() else 0 for item in temp.item_id]
df_ranker_train = df_ranker_train.merge(temp, on='item_id')

#(Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
for week in range(1,53):
    df_ranker_train[f'w_{week}_sub'] = df_ranker_train.apply(lambda x: user_cat_week_sub(week, x[0], x[6], u_i_week_dict), axis=1)

#(Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
for week in range(1,53):
    df_ranker_train[f'w_{week}_sub'] = df_ranker_train.apply(lambda x: user_cat_week_div(week, x[0], x[6], u_i_week_dict), axis=1)

df_ranker_train.head()

Wall time: 6min 34s


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,w_43_sub,w_44_sub,w_45_sub,w_46_sub,w_47_sub,w_48_sub,w_49_sub,w_50_sub,w_51_sub,w_52_sub
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,0.0,0.0,0.000000,0.0,0.000039,0.0,0.000043,0.000008,0.0,0.0
1,1434,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,NaN,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,1174,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,25-34,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,1216,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,NaN,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,900,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,35-44,...,0.0,0.0,0.000105,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [48]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [49]:
cat_feats = X_train.columns[2:15].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

## Обучение модели ранжирования

In [50]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


In [51]:
df_ranker_predict = df_ranker_train.copy()

In [52]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [53]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Eval matching on test dataset

In [54]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

Wall time: 6.61 s


In [55]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.1444117647058813)]

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [56]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [57]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [58]:
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.20616187989555942)
('own_rec', 0.1444117647058813)


C:\Users\Nata\Desktop\webinar_6\webinar_6\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


Берем топ-k предсказаний, ранжированных по вероятности, для каждого юзера

# Домашнее задание

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?